In [1]:
from FunctionSet import *
# The NRPy_param_funcs module sets up global structures that manage free parameters within NRPy+
import NRPy_param_funcs as par 
# The indexedexp module defines various functions for defining and managing indexed quantities like tensors and pseudotensors
import indexedexp as ixp
# The grid module defines various parameters related to a numerical grid or the dimensionality of indexed expressions
# For example, it declares the parameter DIM, which specifies the dimensionality of the indexed expression
import grid as gri
from outputC import *
import numpy as np

import sympy
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal

In [2]:
Nx, Ny, Nz= symbols('Nx Ny Nz', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
name = Idx('name')
dx, dy, dz = symbols('dx dy dz')

var_names = ["Phi","Pi"]

phi = IndexedBase('state_fab')[i,j,k,var_names[0]]
pi = IndexedBase('state_fab')[i,j,k,var_names[1]]

print(phi)
print(pi)

print("Idx::phi")

state_fab[i, j, k, Phi]
state_fab[i, j, k, Pi]
Idx::phi


In [3]:
RHS_dotPHI = pi
RHS_dotPI = Dc2(phi,'x')+Dc2(phi,'y')

In [4]:
RHS_dotPI

(state_fab[i, j - 1, k, Phi] + state_fab[i, j + 1, k, Phi] - 2*state_fab[i, j, k, Phi])/dy**2 + (state_fab[i - 1, j, k, Phi] + state_fab[i + 1, j, k, Phi] - 2*state_fab[i, j, k, Phi])/dx**2

In [5]:
print(AMReXcode(RHS_dotPHI,var_names,True,var_names[0])+"\n")
print(AMReXcode(RHS_dotPI,var_names,True,var_names[1]))

rhs_fab(i, j, k, Idx::Phi) = state_fab(i, j, k, Idx::Pi);

rhs_fab(i, j, k, Idx::Pi) = (state_fab(i, j - 1, k, Idx::Phi) + state_fab(i, j + 1, k, Idx::Phi) - 2*state_fab(i, j, k, Idx::Phi))/(dx[1]*dx[1]) + (state_fab(i - 1, j, k, Idx::Phi) + state_fab(i + 1, j, k, Idx::Phi) - 2*state_fab(i, j, k, Idx::Phi))/(dx[0]*dx[0]);
